In [1]:
import torch 
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import os
from torch import optim
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_sequence
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
import numpy as np

## Parameters Setup

In [2]:
# parameters
MAX_LENGTH = 25
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu") # to be removed
print(device)

SOS_token = 1
EOS_token = 2

hidden_size = 512

batch_size = 128

attention_vector_size = 256

cuda


## Data Setup

In [3]:
# set data path
data_dir = os.path.join('datasets', 'nmt_data_vi')
train_source = 'train.vi'
train_target = 'train.en'
train_source_dir = os.path.join(data_dir, train_source)
train_target_dir = os.path.join(data_dir, train_target)
vocab_source = 'vocab.vi'
vocab_target = 'vocab.en'
vocab_source_dir = os.path.join(data_dir, vocab_source)
vocab_target_dir = os.path.join(data_dir, vocab_target)

In [4]:
# load training sets
with open(train_source_dir) as f_source:
    sentences_source = f_source.readlines()
with open(train_target_dir) as f_target:
    sentences_target = f_target.readlines()

# check the total number of sentencs in training sets    
print("Total number of sentences in source training set: {}".format(len(sentences_source)))
print("Total number of sentences in target training set: {}".format(len(sentences_target)))

Total number of sentences in source training set: 133317
Total number of sentences in target training set: 133317


In [5]:
# check the longest sentence after sentence truncation
max = 0
for s in sentences_source:
    if len(s) > max:
        max = len(s)
        max_s = s
print("Number of words in the longest sentence in sentences_source: {}".format(max))
print("The longest sentence: \n{}".format(max_s))

Number of words in the longest sentence in sentences_source: 3199
The longest sentence: 
Thula Mama , Thula Mama , Thula Mama , Thula Mama . Trong kí ức tuổi thơ con , qua những giọt lệ nhoè mắt bà , con thấy chân lý trong nụ cười của bà , con thấy chân lý trong nụ cười của bà , xuyên thấu màn đêm u tối trong sự vô tri của con . Ôi , có một người bà đang nằm nghỉ bà ốm đau và trái tim bà rơi lệ . Băn khoăn , băn khoăn , băn khoăn , băn khoăn liệu thế giới này đang đi về đâu . Lẽ nào chuyện trẻ nhỏ phải tự xoay xở lấy là đúng ? Không , không , không , không , không , không . Lẽ nào phiền muộn dồn hết lên mái đầu người phụ nữ già là đúng ? Những người vô danh bất hạnh . Thula Mama Mama , Thula Mama . Thula Mama Mama . Thula Mama , Thula Mama , Thula Mama Mama , Thula Mama . Ngày mai sẽ tốt đẹp hơn . Ngày mai trèo đèo lội suối sẽ dễ hơn , bà ơi . Thula Mama , Thula Mama . Tôi có nên tan vào bài hát này như người đàn ông hát nhạc blues hay một người hát rong . Và rồi từ rất xa , không phải

In [6]:
# Truncate sentences by maximum length
sentences_source = list(map(lambda src:src.split()[:MAX_LENGTH], sentences_source))
sentences_target = list(map(lambda src:src.split()[:MAX_LENGTH], sentences_target))

In [7]:
# Delete empty sentences in source and target
i = 0
while i < len(sentences_source):
    if sentences_source[i]==[] or sentences_target[i]==[]:
        del sentences_source[i]
        del sentences_target[i]
        i -= 1
    i += 1
print(len(sentences_source))
print(len(sentences_target))

133166
133166


In [8]:
# load vocabularies

# build index2word
with open(vocab_source_dir) as f_vocab_source:
    #index2word_source = f_vocab_source.readlines()
    index2word_source = [line.rstrip() for line in f_vocab_source]
with open(vocab_target_dir) as f_vocab_target:
    #index2word_target = f_vocab_target.readlines()
    index2word_target = [line.rstrip() for line in f_vocab_target]

# build word2index
word2index_source = {}
for idx, word in enumerate(index2word_source):
    word2index_source[word] = idx
word2index_target = {}
for idx, word in enumerate(index2word_target):
    word2index_target[word] = idx
    
# check vocabularies size    
source_vocab_size = len(index2word_source)
target_vocab_size = len(index2word_target)
print("Total nummber of words in source vocabulary: {}".format(len(index2word_source)))
print("Total nummber of words in target vocabulary: {}".format(len(index2word_target)))    

Total nummber of words in source vocabulary: 7709
Total nummber of words in target vocabulary: 17191


In [9]:
# helper funtions to convert sentence in natural language to list of word indexes
def sen2idx(sentence, word2index):
    return [word2index.get(word, 0) for word in sentence] # assume that 0 is for <unk>

def sen2tensor(sentence, word2index):
    idxes = sen2idx(sentence, word2index)
    idxes.append(EOS_token)
    return torch.tensor(idxes, dtype=torch.long, device=device)

## Token to be ignored

In [10]:
PAD_token = target_vocab_size # padding value

## Batch Generator

In [11]:
class BatchGenerator():
    def __init__(self, batch_size, sentences_source, sentences_target, word2index_source, word2index_target):
        self.batch_size = batch_size
        self.sentences_source = sentences_source
        self.sentences_target = sentences_target
        self.word2index_source = word2index_source
        self.word2index_target = word2index_target
        self.num_sentence = len(sentences_source)
        self.reset()
    
    def reset(self):
        self.consumed = 0
        self.permutation = np.random.permutation(self.num_sentence)
    
    def get_batch(self):
        # generate id in one batch
        if self.consumed + self.batch_size > self.num_sentence:
            self.reset()
        sample_id = self.permutation[self.consumed:self.consumed + self.batch_size]
        self.consumed += self.batch_size

        #generate a source batch
        sentences_source_tensor = [sen2tensor(self.sentences_source[id], self.word2index_source) for id in sample_id]

        len_array_source = [len(st) for st in sentences_source_tensor]
        reorder_idx_source = np.argsort(len_array_source, kind='mergesort')
        reorder_idx_source = np.argsort(np.flip(reorder_idx_source, 0)) #index to restore unsorted order

        sentences_source_tensor.sort(key=len)
        sentences_source_tensor.reverse()
        sentences_source_packed = pack_sequence(sentences_source_tensor)

        #generate a target batch
        sentences_target_tensor = [sen2tensor(self.sentences_target[id], self.word2index_target) for id in sample_id]

        len_array_target = [len(st) for st in sentences_target_tensor]
        reorder_idx_target = np.argsort(len_array_target, kind='mergesort')
        reorder_idx_target = np.argsort(np.flip(reorder_idx_target, 0)) #index to restore unsorted order

        sentences_target_tensor.sort(key=len)
        sentences_target_tensor.reverse()
        sentences_target_packed = pack_sequence(sentences_target_tensor)

        return (sentences_source_packed, reorder_idx_source), (sentences_target_packed, reorder_idx_target)

In [12]:
# to be depreciated

# def sentences2tensor(sentences, word2index):
#     sentences_tensor = [sen2tensor(s, word2index) for s in sentences]
#     sentences_tensor.sort(key=len, reverse=True)
#     output = pad_sequence(sentences_tensor, batch_first=True)
#     return output

def batch_generator(batch_size, sentences_source, sentences_target, word2index_source, word2index_target):
    #output: two PackedSequence object, two indexes to reorder sentences in a batch
    
    # generate id in one batch
    total = len(sentences_source)
    sample_id = np.random.choice(total, batch_size, replace=False)
    
    #generate a source batch
    sentences_source_tensor = [sen2tensor(sentences_source[id], word2index_source) for id in sample_id]
    
    len_array_source = [len(st) for st in sentences_source_tensor]
    reorder_idx_source = np.argsort(len_array_source, kind='mergesort')
    reorder_idx_source = np.argsort(np.flip(reorder_idx_source, 0)) #index to restore unsorted order
    
    sentences_source_tensor.sort(key=len)
    sentences_source_tensor.reverse()
    sentences_source_packed = pack_sequence(sentences_source_tensor)
    
    #generate a target batch
    sentences_target_tensor = [sen2tensor(sentences_target[id], word2index_target) for id in sample_id]
    
    len_array_target = [len(st) for st in sentences_target_tensor]
    reorder_idx_target = np.argsort(len_array_target, kind='mergesort')
    reorder_idx_target = np.argsort(np.flip(reorder_idx_target, 0)) #index to restore unsorted order
    
    sentences_target_tensor.sort(key=len)
    sentences_target_tensor.reverse()
    sentences_target_packed = pack_sequence(sentences_target_tensor)
    
    return (sentences_source_packed, reorder_idx_source), (sentences_target_packed, reorder_idx_target)

In [13]:
# test batch_generator
a = ['I','am','a','boy']
b = ['a']
c = ['the','goat']

sentences_test = [a,b,c]
sentences_test2 = [c,b,a]

BG_test = BatchGenerator(2, sentences_test, sentences_test2, word2index_target, word2index_target)

(output_test, reorder_idx_test), (output_test2, reorder_idx_test2) = BG_test.get_batch()
print(output_test)
print(reorder_idx_test)
print(pad_packed_sequence(output_test, batch_first=True))

del a, b, c, sentences_test, sentences_test2, output_test, reorder_idx_test, output_test2, reorder_idx_test2

PackedSequence(data=tensor([  47,    8,  382,    2,    8,  897,    2], device='cuda:0'), batch_sizes=tensor([ 2,  2,  1,  1,  1]))
[0 1]
(tensor([[  47,  382,    8,  897,    2],
        [   8,    2,    0,    0,    0]], device='cuda:0'), tensor([ 5,  2]))


## Helper function to resume order of sentences in a batch

Order recovery is necessary because sentences have to be sorted in descending order of sentence length to be packed as a PackedSequence object. PackedSequence object helps to deal with inputs with variable length in NMT setting. LSTM, RNN can accept PackedSequence objects.

In [14]:
def resume_order(input, idx):
    # input 
    #   input: Tensor: (batch_size, seq_length)
    #   idx: Tensor or ndarray: (batch_size)
    # output
    #   out: Tensor with reordered sentences in batch: (batch_size, seq_length) 
    
    if isinstance(idx, (np.ndarray)):
        idx = torch.from_numpy(idx)
        if device == torch.device("cuda"):
            idx = idx.cuda()
    out = torch.index_select(input, 0, idx)
    return out

In [15]:
# test resume_order
input_test = torch.tensor([[1,2,3],[4,5,6],[7,8,9]],device=device)
idx_test = np.array([2,1,0])
print(resume_order(input_test, idx_test))
input_test = torch.tensor([[1,2,3],[4,5,6],[7,8,9]],device=device)
idx_test = torch.tensor([2,1,0], device=device)
print(resume_order(input_test, idx_test))
print(idx_test.device)
del input_test, idx_test

tensor([[ 7,  8,  9],
        [ 4,  5,  6],
        [ 1,  2,  3]], device='cuda:0')
tensor([[ 7,  8,  9],
        [ 4,  5,  6],
        [ 1,  2,  3]], device='cuda:0')
cuda:0


## Define Encoder and Decoder classes

In [16]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, num_layers=1, num_directions=1, dropout=0):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.num_directions = num_directions
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
    def forward(self, input_tuple, prev_h, prev_c):
        # input
        # input size: (batch_size, seq_length)
        # prev_h size: (num_layers*num_directions, batch_size, hidden_size)
        # prec_c size: (num_layers*num_directions, batch_size, hidden_size)
        # output
        # h_n size: (num_layers*num_directions, batch_size, hidden_size)
        # c_n size: (num_layers*num_directions, batch_size, hidden_size)
        (sentences_packed, reorder_idx) = input_tuple
        sentences_tensor, sentences_length = pad_packed_sequence(sentences_packed, batch_first=True, padding_value=0)
        #sentences_tensor: (batch_size, seq_length)
        input_embedded = self.embedding(sentences_tensor) # (batch_size, seq_length, hidden_size)
        input_embedded_packed = pack_padded_sequence(input_embedded, sentences_length, batch_first=True)
        output, (h_n, c_n) = self.lstm(input_embedded_packed, (prev_h, prev_c))
        return output, h_n, c_n
    def initHidden(self):
        return torch.zeros(self.num_layers*self.num_directions, self.batch_size, self.hidden_size, device=device)

In [17]:
# num_layers and num_directions for encoder must be 0 when this decoder is used
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size, batch_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTMCell(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input, prev_h, prev_c, encoder_output=None):
        input_embedded = self.embedding(input)
        h, c = self.lstm(input_embedded, (prev_h, prev_c))
        output = self.softmax(self.out(h))
        return output, h, c
    def initHidden(self):
        return torch.zeros(self.batch_size, self.hidden_size, device=device)

In [18]:
class DotAttenDecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size, batch_size):
        super(DotAttenDecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTMCell(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size*2, attention_vector_size)
        self.out2 = nn.Linear(attention_vector_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input, prev_h, prev_c, encoder_output):
        # encoder_output: PackedSequence to be converted to (batch_size, seq_length, hidden_size*num_directions)
        input_embedded = self.embedding(input)
        h, c = self.lstm(input_embedded, (prev_h, prev_c))
        
        # trick: use padding value = -inf to do variable length attention correctly 
        encoder_output, _ = pad_packed_sequence(encoder_output, batch_first=True, padding_value=0)
        #print(encoder_output) # to be removed
        scores = torch.matmul(encoder_output, h.unsqueeze(-1)) # (batch_size, seq_length, 1)
        scores[scores==0] = -10e10
        #print(scores) # to be removed
        scores = F.softmax(scores, dim=1)
        context_vector = torch.matmul(torch.transpose(encoder_output, 1, 2), scores).squeeze(-1) # (batch_size, hidden_size)
        attention_vector = F.tanh(self.out(torch.cat((context_vector, h), -1))) # (batch_size, attention_vector_size)
        output = self.softmax(self.out2(attention_vector))
        return output, h, c
        
    def initHidden(self):
        return torch.zeros(self.batch_size, self.hidden_size, device=device)

In [19]:
# This DecoderLSTM can't do teacher forcing in training

# class DecoderLSTM(nn.Module):
#     def __init__(self, hidden_size, output_size, batch_size, num_layers=1, num_directions=1, dropout=0):
#         super(DecoderLSTM, self).__init__()
#         self.hidden_size = hidden_size
#         self.batch_size = batch_size
#         self.num_layers = num_layers
#         self.num_directions = num_directions
#         self.dropout = dropout
        
#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
#         self.out = nn.Linear(hidden_size*num_directions, output_size)
#         self.softmax = nn.LogSoftmax(dim=2)
#     def forward(self, input, prev_h, prev_c):
#         # input 
#         # input size: (batch_size, seq_length)
#         # prev_h size: (num_layers*num_directions, batch_size, hidden_size)
#         # prec_c size: (num_layers*num_directions, batch_size, hidden_size)
#         # output
#         # h_n size: (num_layers*num_directions, batch_size, hidden_size)
#         # c_n size: (num_layers*num_directions, batch_size, hidden_size)
#         # output size: (batch_size, seq_length, output_size)
#         input_embedded = self.embedding(input)
#         output, (h_n, c_n) = self.lstm(input_embedded, (prev_h, prev_c))
#         output =self.softmax(self.out(h))
#         return output, h_n, c_n
#     def initHidden(self, batch_size):
#         return torch.zeros(self.num_layers*self.num_directions, self.batch_size, self.hidden_size, device=device)

## Training

In [20]:
def train(source_tuple, target_tuple, encoder, decoder, encoder_optimizer, decoder_optimizer,batch_size, max_length=MAX_LENGTH):
    
    encoder_hidden_h = encoder.initHidden()
    encoder_hidden_c = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0
    
    criterion = nn.NLLLoss(ignore_index=PAD_token, size_average=False)
    
    (sentences_source_packed, reorder_idx_source) = source_tuple
    (sentences_target_packed, reorder_idx_target) = target_tuple
    sentences_target_tensor, sentences_target_length = pad_packed_sequence(sentences_target_packed, batch_first=True, padding_value=PAD_token)
    sentences_target_tensor = resume_order(sentences_target_tensor, reorder_idx_target)

    target_length = sentences_target_tensor.size(1)
    
    # encoder_output size: (batch_size, seq_length, hidden_size*num_directions)
    # encoder_hidden_h size: (num_layers*num_directions, batch_size, hidden_size)
    # encoder_hidden_c size: (num_layers*num_directions, batch_size, hidden_size)
    encoder_output, encoder_hidden_h, encoder_hidden_c = encoder(source_tuple, encoder_hidden_h, encoder_hidden_c)
    
    
    decoder_input = torch.full((batch_size,), SOS_token, dtype=torch.long, device=device)
    decoder_hidden_c = resume_order(encoder_hidden_c[0], reorder_idx_source)
    decoder_hidden_h = resume_order(encoder_hidden_h[0], reorder_idx_source)
    
    to_print = []
    
    for di in range(target_length):
        decoder_output, decoder_hidden_h, decoder_hidden_c = decoder(decoder_input, decoder_hidden_h, decoder_hidden_c, encoder_output)
        loss += criterion(decoder_output, sentences_target_tensor[:,di])
        decoder_input = sentences_target_tensor[:,di]
        
        decoder_output_np = np.argmax(decoder_output.detach().cpu().numpy(), 1)[0]
        to_print.append(index2word_target[decoder_output_np])
    
    denominator = torch.sum(sentences_target_length).float()
    if device == torch.device("cuda"):
        denominator = denominator.cuda()
    loss = loss / denominator
    
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item(), to_print

In [21]:
def trainIters(batch_generator, encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.1):
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    encoder_optimizer = optim.Adam(encoder.parameters(), learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), learning_rate)
    
    for iter in range(1, n_iters+1):
        source_tuple, target_tuple = batch_generator.get_batch()
        loss, to_print = train(source_tuple, target_tuple, encoder, decoder, encoder_optimizer, decoder_optimizer, batch_size)
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter%print_every ==0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('(%d %d%%) %.4f' % (iter, iter / n_iters * 100, print_loss_avg))
            print(to_print)

In [22]:
batch_generator1 = BatchGenerator(batch_size, sentences_source, sentences_target, word2index_source, word2index_target)
encoder1 = EncoderLSTM(source_vocab_size, hidden_size, batch_size).to(device)
decoder1 = DecoderLSTM(hidden_size, target_vocab_size+1, batch_size).to(device) # +1 is a wordaround for ignore_index field of NLLLoss
trainIters(batch_generator1, encoder1, decoder1, 12000, print_every=100, learning_rate = 0.001)
torch.save(encoder1.state_dict(), os.path.join("model_ckpt","model_maxlen25_noatten_lr0.001_step12000_encoder"))
torch.save(decoder1.state_dict(), os.path.join("model_ckpt","model_maxlen25_noatten_lr0.001_step12000_decoder"))

(100 0%) 5.9977
['And', 'I', '&apos;m', 'to', 'the', ',', 'the', 'of', ',', ',', 'and', 'is', 'a', '<unk>', '.', 'the', '<unk>', '.', '.', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']
(200 1%) 5.0734
['And', ',', 'the', ',', ',', 'I', 'was', 'got', 'to', 'little', 'to', ',', ',', 'the', 'the', '<unk>', '.', '.', '.', 'the', '.', 'the', '.', '</s>', '</s>', '</s>']
(300 2%) 4.8125
['It', 'are', 'the', ',', 'the', ',', '<unk>', 'of', ',', 'and', 'it', 'can', '&apos;t', 'that', 'the', '&apos;s', 'not', 'to', 'the', ',', 'the', ',', '.', '<unk>', '</s>', '</s>']
(400 3%) 4.6265
['You', 'you', 'can', 'see', 'you', '<unk>', ',', 'but', '&apos;s', 'a', '<unk>', 'of', 'but', 'it', 'you', 'it', '&apos;t', 'be', '.', 'but', 'can', '&apos;t', 'be', '.', '.', '</s>']
(500 4%) 4.4876
['&quot;', 'It', '&apos;m', 'a', '<unk>', 'the', 'of', ',', ',', '&quot;', 'it', 'is', '&apos;t', 'know', 'like', 'this', '.', 'the', '.', '<unk>', '.', '</s>', '</s>', '</s>', '</s>', '</s>']
(600 5%) 4.368

(4500 37%) 2.7344
['It', 'it', 'seems', 'seem', 'to', 'a', 'simple', 'idea', ',', 'but', 'it', '&apos;s', ',', 'but', 'it', 'idea', 'that', 'has', 'be', 'to', 'the', '&apos;s', 'thoughts', '.', 'be', '</s>']
(4600 38%) 2.7450
['These', 'is', 'the', 'creatures', 'because', 'because', '<unk>', ',', 'because', 'a', ',', 'because', 'they', '<unk>', 'animal', 'are', 'have', 'a', '<unk>', 'nickname', '.', '</s>', '.', '.', '.', '</s>']
(4700 39%) 2.7720
['<unk>', 'percent', 'of', 'people', 'have', 'that', '<unk>', 'their', '<unk>', ',', 'they', '&apos;re', 'have', 'a', 'percent', 'of', 'people', ',', '</s>', ',', 'to', 'and', ',', '</s>', '</s>', '</s>']
(4800 40%) 2.7765
['In', '&apos;re', 'very', 'very', 'different', '.', 'They', '&apos;re', 'very', '<unk>', 'different', '.', '.', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
(4900 40%) 2.7689
['The', '<unk>', 'station', 'are', 'out', '<unk>', '.', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.

(8700 72%) 1.9870
['Anyway', 'all', ',', 'it', '&apos;s', 'no', '<unk>', 'or', 'jumping', ',', 'and', 'the', '<unk>', 'is', 'up', '.', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.']
(8800 73%) 2.0042
['And', 'the', 'great', 'thing', 'is', ',', 'if', 'those', 'numbers', 'are', 'different', 'value', 'values', 'or', 'compared', 'number', 'size', ',', 'or', 'number', 'is', 'as', 'compared', 'knew', 'it', '</s>']
(8900 74%) 2.0267
['Because', 'he', 'saw', 'staring', ',', 'was', 'front', '<unk>', 'diner', 'suit', ',', 'and', 'mustache', ',', 'staring', 'at', 'me', ',', '</s>', '.', '.', '.', '.', '</s>', '</s>', '</s>']
(9000 75%) 2.0430
['The', 'change', 'changed', 'the', 'children', 'have', 'changed', '.', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
(9100 75%) 2.0564
['It', 'has', 'got', 'a', 'sensors', ',', 'the', 'motors', ',', ',', 'from', ',', 'of', 'the', 'autonomy', ',', ',', 'and', 'it', 'than', 'a', 'kilos', 'of', 'the', 'in', 

In [23]:
trainIters(batch_generator1, encoder1, decoder1, 12000, print_every=100, learning_rate = 0.001)
torch.save(encoder1.state_dict(), os.path.join("model_ckpt","model_maxlen25_noatten_lr0.001_step24000_encoder"))
torch.save(decoder1.state_dict(), os.path.join("model_ckpt","model_maxlen25_noatten_lr0.001_step24000_decoder"))

(100 0%) 1.7033
['I', 'start', 'with', 'the', 'ideas', '.', 'I', 'mean', 'it', '.', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '</s>', '</s>', '</s>', '</s>']
(200 1%) 1.7319
['Now', 'now', 'I', '&apos;m', 'going', 'to', 'try', 'the', 'from', 'pulse', 'to', 'my', 'brain', ',', 'and', 'my', 'brain', 'of', 'my', 'brain', '.', 'controls', 'your', 'brain', '.', '</s>']
(300 2%) 1.7440
['So', 'now', 'we', 'go', 'to', ',', 'I', 'want', 'to', 'show', 'you', 'that', 'it', 'it', 'means', 'in', 'if', 'the', '.', '</s>', 'to', '.', '.', '.', '.', '</s>']
(400 3%) 1.7367
['Because', 'the', 'brain', 'of', 'the', 'brain', 'that', 'controls', 'decision-making', ',', '&apos;t', 'control', 'the', '.', '</s>', 'decisions', '.', '.', '.', '.', '.', '.', '.', '</s>', '</s>', '</s>']
(500 4%) 1.6890
['And', 'he', '&apos;s', 'use', 'this', 'new', 'and', 'and', 'it', 'it', 'quite', 'extraordinary', '.', '</s>', '?', '?', '?', '.', '.', '.', '.', '.', '.', '.', '.', '</s>']
(600 5%) 1.4422

(4300 35%) 1.3370
['In', 'addition', 'to', 'the', '<unk>', 'generated', 'the', 'the', 'charcoal', 'is', 'about', 'million', 'dollars', '.', '</s>', '.', '.', '.', '.', '.', '</s>', '</s>', '</s>', '</s>', '</s>', '</s>']
(4400 36%) 1.3600
['So', 'this', 'this', 'woman', ',', 'Mrs.', '<unk>', 'Han', ',', 'I', 'suppose', ',', 'be', 'taught', 'me', 'to', 'most', 'powerful', 'they', 'have', 'learned', 'about', '.', '.', '</s>', '</s>']
(4500 37%) 1.3570
['So', 'here', '&apos;s', 'the', ':', ':', '<unk>', 'interdependence', '.', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
(4600 38%) 1.3834
['And', 'these', 'genes', 'don', 'it', 'situation', 'to', 'just', 'as', 'a', 'as', 'a', 'gene', 'gene', '.', '.', 'acted', '.', '</s>', '.', '.', '.', '.', '.', '.', '.']
(4700 39%) 1.2235
['And', 'if', 'you', 'think', 'about', 'this', 'this', 'experiment', 'testing', ',', 'in', 'order', 'introducing', 'context', 'elements', 'of', 'randomness', 'in', 'the', 'the

(8400 70%) 1.0792
['We', 'created', 'an', '<unk>', 'of', 'a', 'flash', 'of', 'insight', ',', '&quot;', 'stroke', 'of', 'insight', ',', 'we', 'have', ',', ',', 'or', 'call', '&quot;', 'eureka', '!', '&quot;', '</s>']
(8500 70%) 1.0983
['I', 'work', 'done', 'doing', 'this', 'for', 'the', 'last', '15', 'years', ';', 'my', '&apos;s', 'my', 'job', 'job', ',', 'and', 'you', 'want', 'to', '</s>', '.', '.', '.', '.']
(8600 71%) 1.1228
['Mac', 'Barnett', ':', 'Why', 'a', 'book', 'book', 'is', 'a', 'secret', 'door', '</s>', '</s>', '.', '.', '?', '?', '?', '?', '?', '?', '?', '</s>', '</s>', '</s>', '</s>']
(8700 72%) 1.1294
['Look', ',', 'I', '&apos;m', 'go', 'around', 'and', 'say', ',', 'of', '.', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
(8800 73%) 1.1449
['Country', 'A', ',', 'in', '1990', ',', 'had', 'about', '$', '300', 'per', 'capita', 'GDP', 'as', 'compared', 'with', 'Country', 'B', ',', 'which', 'in', '$', '<unk>', 'per', 'America', '</s>']
(8900 74%)

In [24]:
trainIters(batch_generator1, encoder1, decoder1, 12000, print_every=100, learning_rate = 0.001)
torch.save(encoder1.state_dict(), os.path.join("model_ckpt","model_maxlen25_noatten_lr0.001_step36000_encoder"))
torch.save(decoder1.state_dict(), os.path.join("model_ckpt","model_maxlen25_noatten_lr0.001_step36000_decoder"))

(100 0%) 0.8351
['He', 'said', ',', '&quot;', '<unk>', '&apos;s', 'Billy', 'Graham', ',', 'the', 'preacher', '.', '&quot;', '</s>', '.', '.', '.', '.', '.', '.', '.', '</s>', '</s>', '</s>', '</s>', '</s>']
(200 1%) 0.8563
['So', ',', 'what', 'do', 'it', 'to', '?', '</s>', '.', '?', '?', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', 'to', 'to', 'to', 'to', 'to']
(300 2%) 0.8698
['There', 'people', 'are', 'cooking', 'fires', 'in', 'the', 'cooking', ',', 'whether', 'it', '&apos;s', 'camel', 'dung', 'or', 'wood', '.', '</s>', 'photos', '.', '.', '.', '.', '.', '.', '.']
(400 3%) 0.8958
['Up', 'to', 'the', 'around', 'about', 'war', 'Second', 'World', 'War', ',', '<unk>', '<unk>', 'invariably', 'dictators', '--', 'these', '<unk>', 'figures', 'who', 'were', 'rehearse', ',', 'not', 'just', 'the', '</s>']
(500 4%) 0.9070
['And', 'indeed', ',', 'that', '&apos;s', 'exactly', 'what', 'this', '.', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '</s>', '</s>']
(60

(4100 34%) 0.8380
['So', ',', 'in', 'the', 'same', 'way', 'that', 'you', 'wake', 'up', ',', 'if', 'a', 'shower', 'and', 'get', 'dressed', ',', 'you', 'have', 'to', 'be', 'to', 'them', 'that', '</s>']
(4200 35%) 0.6910
['Now', ',', 'the', 'the', 'people', 'that', 'could', '&quot;', 'them', 'apart', ',', 'when', 'the', 'labels', 'were', 'off', ',', 'they', 'were', '&quot;', '<unk>', ',', '&quot;', 'and', 'when', '</s>']
(4300 35%) 0.7045
['So', 'they', 'were', 'happy', 'with', 'it', '.', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
(4400 36%) 0.7194
['That', 'guides', 'you', 'back', 'to', 'the', 'location', 'that', 'you', 'want', 'to', 'remember', '.', '</s>', '.', '?', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
(4500 37%) 0.7416
['It', '&apos;s', 'very', 'different', 'than', 'the', 'different', 'cultures', '.', '&quot;', 'right', '?', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
(4600 38%) 0.7518
['T

(8200 68%) 0.7733
['Lord', '<unk>', 'once', 'said', 'he', 'saw', 'people', 'bathing', 'in', 'the', 'sky', 'Sea', ',', 'and', 'he', 'said', ',', '&quot;', 'Why', 'don', 'you', 'one', 'on', 'you', 'what', '</s>']
(8300 69%) 0.6685
['And', 'it', '&apos;s', 'the', 'we', 'can', 'bend', 'the', 'arc', 'of', 'history', 'down', 'towards', 'zero', ',', 'just', 'doing', 'the', 'things', 'that', 'we', 'know', 'or', '.', '</s>', '</s>']
(8400 70%) 0.5997
['And', 'I', 'said', ',', '&quot;', 'What', '&apos;s', 'your', 'Movember', 'story', '?', '&quot;', '</s>', '.', '.', '.', '.', '.', '.', '.', 'differently', 'differently', 'to', 'to', 'to', '.']
(8500 70%) 0.6211
['One', '<unk>', 'of', 'this', 'is', 'in', 'the', 'United', 'Nations', '.', '</s>', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
(8600 71%) 0.6382
['Actually', 'actually', '&apos;s', 'actually', 'a', 'instructions', 'that', 'you', 'only', 'use', 'once', 'a', 'you', 'you', '.', '</s>', '.', '.', '.', '.', '.', 